In [1]:
import torch 

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

# Functions

In [ ]:
def Load_images(img,)

# Hyperperameters

In [3]:
num_epochs = 15
num_classes = 2
learning_rate = 0.001

True

In [ ]:
#